# Select reference samples
- 2020.12.29
- Idea：两两样本计算关联系数，填到一个三角阵中。方便后期调用。
- 尚未成功

In [85]:
import os
import numpy as np
import function as df
import itertools
from datetime import datetime 

In [44]:
sample_list_file='/home/rt2776/cnv_espresso/source/spark_sample_27270.txt'
sample_rd_dir='/home/rt2776/cnv_espresso/data/norm'

In [45]:
sample_list = df.fileToList(sample_list_file)

In [49]:
sample_list[0:5]
sample_list[3]

'SP0000034'

In [5]:
RD_matrix = np.zeros([len(sampleID), len(sampleID)], dtype=np.float)

In [6]:
RD_matrix.shape

(27270, 27270)

In [19]:
comb_tuple = itertools.combinations(sampleID, 2)

In [27]:
type(comb_tuple)

itertools.combinations

In [28]:
num = 0
sample_comb_list = []
for comb in comb_tuple:
    num += 1
    sample_comb_list.append(comb)
    print(num,comb,type(comb))

In [31]:
sample_comb_list[1:5]

[('SP0000002', 'SP0000027'),
 ('SP0000002', 'SP0000034'),
 ('SP0000002', 'SP0000051'),
 ('SP0000002', 'SP0000063')]

In [39]:
case_sample = sample_list[0]
case_sample_rd_file = sample_rd_dir + '/'+ case_sample +'.cov.bed.norm.bz2'
case_sample_rd_file

'/home/rt2776/cnv_espresso/data/norm/SP0000002.cov.bed.norm.bz2'

In [50]:
ref_sample = sample_list[1]
ref_sample_rd_file = sample_rd_dir + '/'+ ref_sample +'.cov.bed.norm.bz2'
ref_sample_rd_file

'/home/rt2776/cnv_espresso/data/norm/SP0000003.cov.bed.norm.bz2'

In [51]:
rc = np.loadtxt(case_sample_rd_file, dtype=np.float, delimiter="\t", skiprows=0, usecols=[5])
rc_ref = np.loadtxt(ref_sample_rd_file, dtype=np.float, delimiter="\t", skiprows=0, usecols=[5])

In [52]:
rc
rc_ref

array([0.301659, 0.      , 0.      , ..., 0.      , 0.      , 0.      ])

In [70]:
corr = np.corrcoef(rc,rc_ref)

In [96]:
a=datetime.now() 
num = 0
for sampleID in sample_list[0:100]:
    num += 1
    sample_rd_file = sample_rd_dir + '/'+ sampleID +'.cov.bed.norm.bz2'
    rc = np.loadtxt(sample_rd_file, dtype=np.float, delimiter="\t", skiprows=0, usecols=[5])
    if not os.path.exists(sample_rd_file):
        file_condition = "False"
    else:
        file_condition = "Exist"
    if(num%10==0):
        print(num, sampleID,file_condition,rc)
print(num)
b=datetime.now() 
(b-a).seconds

10 SP0000071 Exist [0. 0. 0. ... 0. 0. 0.]
20 SP0000140 Exist [0.158682 0.       0.       ... 0.       0.       0.      ]
30 SP0001282 Exist [0.15276 0.      0.      ... 0.      0.      0.     ]
40 SP0001661 Exist [0.206773 0.       0.       ... 0.       0.       0.      ]
50 SP0000349 Exist [0. 0. 0. ... 0. 0. 0.]
60 SP0000494 Exist [0.273135 0.       0.       ... 0.       0.       0.      ]
70 SP0013406 Exist [0.003803 0.       0.       ... 0.       0.       0.      ]
80 SP0000882 Exist [0.203371 0.       0.       ... 0.       0.       0.      ]
90 SP0001059 Exist [0. 0. 0. ... 0. 0. 0.]
100 SP0001040 Exist [0.175686 0.       0.       ... 0.       0.       0.      ]
100


93

打开100个文件需要93秒（不包括cor的计算），即1秒中1个文件，那么27270个样本两两配对计算三角阵 `sum = n*(1+n)/2=27270*(1+27270)/2=371840085`次计算；
371840085秒/100节点/3600小时/24天=43天，太慢了！

In [10]:
np.save("RD_matrix",RD_matrix)